In [120]:
from scipy import integrate, constants, optimize
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [121]:
Temp = 300

In [122]:
class Thermodynamic:
    def __init__(self, name: str):
        cols = ['Compound', '$\\Delta H_f$', '$S_f$', 'A', 'B', 'C', 'D', '$T_1$', '$T_2$']
        data = pd.read_csv('../../unit-04/ex-3/test-tab-04.csv', usecols=cols)
        compound_data = data.loc[data['Compound'] == name]
        self.Name = name
        self.Delta_H = float(compound_data['$\\Delta H_f$'].values[0])
        self.Delta_S = float(compound_data['$S_f$'].values[0]) * 1e-3
        self.A = float(compound_data['A'].values[0])
        self.B = float(compound_data['B'].values[0])
        self.C = float(compound_data['C'].values[0])
        self.D = float(compound_data['D'].values[0])
        self.T_min = float(compound_data['$T_1$'].values[0])
        self.T_max = float(compound_data['$T_2$'].values[0])

    def heat_capacity(self, T: float) -> float:
        return 1e-3 * (self.A + self.B * 1e-3 * T + self.C * 1e5 * np.pow(T, -2) + self.D * 1e-6 * np.pow(T, 2))

    def enthalpy(self, T: float):
        return self.Delta_H + integrate.quad(self.heat_capacity, 298, T)[0]

    def entropy(self, T: float):
        return self.Delta_S + integrate.quad(lambda x: self.heat_capacity(x) / x, 298, T)[0]

    def gibbs_energy(self, T: float):
        return self.enthalpy(T) - T * self.entropy(T)

In [123]:
species = ["Methane", "Hydrogen", "Oxygen", "Water", "Carbon Monoxide", "Carbon Dioxide"]
compounds = [Thermodynamic(name) for name in species]

In [124]:
Gjo = np.array([compound.gibbs_energy(Temp) for compound in compounds])
Gjo

array([-130.51063913,  -39.20389448,  -61.54489676, -298.47582504,
       -169.83949514, -457.63594876])

In [125]:
lgP = 1
def func(nj):
    nj = np.array(nj)
    Enj = np.sum(nj)
    G = np.sum(nj * (Gjo / constants.R / Temp + lgP + np.log(nj / Enj)))
    return G

In [126]:
Aeq = np.array(
    [
        [4,   2,   0,   2,   0,   0], # hydrogen balance
        [0,   0,   2,   1,   1,   2], # oxygen balance
        [1,   0,   0,   0,   1,   1], # carbon balance
#        CH4  H2   O2   H2O  CO   CO2
    ])

In [127]:
rng = np.random.default_rng()
moles = 5 * rng.random(6)
beq = np.array([np.sum(moles * Aeq[i]) for i in range(len(Aeq))])

In [128]:
def ec1(n):
    'Equality constraint'
    return np.dot(Aeq, n) - beq

def ic1(n):
    'Inequality constraint: all n>=0'
    return n

In [129]:
n0 = [1, 1, 1, 1, 1, 1] # init guess

In [130]:
X = optimize.fmin_slsqp(func, n0, f_eqcons=ec1, f_ieqcons=ic1, iter=100, acc=1e-6, full_output=1)
X

Optimization terminated successfully    (Exit mode 0)
            Current function value: -12.03663259409686
            Iterations: 8
            Function evaluations: 56
            Gradient evaluations: 8


(array([1.26088029, 2.99302749, 2.66190581, 2.3223202 , 2.91029693,
        2.28390463]),
 np.float64(-12.03663259409686),
 8,
 0,
 'Optimization terminated successfully')